<a href="https://colab.research.google.com/github/aboelela924/siameseCovid19/blob/master/siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import numpy as np
import functools
import cv2 
import os
from random import randint,sample
from tensorflow.keras.layers import Lambda
from sklearn.utils import class_weight
from imutils import build_montages
from sklearn import svm
from tensorflow.keras.applications.densenet import DenseNet169
from sklearn.metrics import accuracy_score


In [ ]:
from tensorflow.keras.models import load_model
from imutils.paths import list_images
import argparse
from tensorflow.keras.layers import Input
from tensorflow.keras import layers

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:

BASE_OUTPUT = "/content/drive/MyDrive/DL"

con_MODEL_PATH = os.path.sep.join([BASE_OUTPUT,
    "contrastive_siamese_model"])
con_PLOT_PATH = os.path.sep.join([BASE_OUTPUT,
    "contrastive_plot.png"])

con_MODEL_PATH_ = os.path.sep.join([BASE_OUTPUT,
    "contrastive_siamese_model_"])
con_PLOT_PATH_ = os.path.sep.join([BASE_OUTPUT,
    "contrastive_plot_.png"])

tri_MODEL_PATH = os.path.sep.join([BASE_OUTPUT,
    "triplet_siamese_model_"])
tri_PLOT_PATH = os.path.sep.join([BASE_OUTPUT,
    "triplet_plot_.png"])

In [ ]:
!kaggle datasets download -d xhlulu/densenet-keras

!unzip -qq /content/densenet-keras.zip

Network_Weight="/content/DenseNet-BC-169-32-no-top.h5"


In [ ]:
IMG_SHAPE = (224,224,3)
features=50
pre_trained_model = DenseNet169(input_shape = IMG_SHAPE, 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(Network_Weight)
for layer in pre_trained_model.layers:
    layer.trainable = False  #to make the layers to Freeze Weights
pre_trained_model.summary()
flatten = tf.keras.layers.Flatten()(pre_trained_model.layers[-2].output)
dense1 = tf.keras.layers.Dense(125, activation='relu')(flatten)
dense1 = tf.keras.layers.BatchNormalization()(dense1)
drop = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(75, activation="relu")(drop)
dense2 = tf.keras.layers.BatchNormalization()(dense2)
new_output = tf.keras.layers.Dense(features)(dense2)
  
featureExtractor = tf.keras.Model(inputs = pre_trained_model.input,outputs=new_output)
featureExtractor.summary()

In [ ]:
class DataGen(tf.keras.utils.Sequence):
    
    def __init__(self, path,
                 batch_size,
                 image_size,
                 exa,
                 loss_type='contrastive'
                 ):
        
        self.path = path
        self.batch_size = batch_size
        self.image_size=image_size
        self.loss_type=loss_type
        self.exa=exa
        self.classes= self.list_full_paths(path)
    
    def list_full_paths(self,directory):
      return [os.path.join(directory, file) for file in os.listdir(directory)]
    def get_img(self,path):
      image=np.asarray(cv2.imread(path))/255
      if image.ndim==2:
        image=np.stack((image,)*3, axis=-1)


      rand_aug=randint(0,2)


      if rand_aug==0:
        pass
      elif rand_aug==1:
        image=np.fliplr(image)
      else:
        image=np.flipud(image)


      return cv2.resize(image, dsize=(self.image_size,self.image_size))

    def __get_in_out(self):

        label=randint(0,1)        
        if self.loss_type =='Triplet':
          pos_neg=sample(self.classes, 2)  

          list_pos=self.list_full_paths(pos_neg[0])
          anc_pos=sample(list_pos, 2)

          list_neg=self.list_full_paths(pos_neg[1])
          neg=sample(list_neg, 1)
          
          return self.get_img(anc_pos[0]),self.get_img(anc_pos[1]),self.get_img(neg[0])

        else:
          if label==0:
            pos_neg=sample(self.classes, 2)

            list_pos=self.list_full_paths(pos_neg[0])
            pos=sample(list_pos, 1)

            list_neg=self.list_full_paths(pos_neg[1])
            neg=sample(list_neg, 1)

            return self.get_img(pos[0]),self.get_img(neg[0]),label

          else:
            pos=sample(self.classes,1 )

            list_pos=self.list_full_paths(pos[0])
            pos=sample(list_pos, 2)

            return self.get_img(pos[0]),self.get_img(pos[1]),label
            


    def __get_data(self):
        if self.loss_type =='Triplet':    
          x_1_batch=np.empty((self.batch_size,self.image_size,self.image_size,3))
          x_2_batch=np.empty((self.batch_size,self.image_size,self.image_size,3))
          x_3_batch=np.empty((self.batch_size,self.image_size,self.image_size,3))
          y_batch=np.zeros((self.batch_size))
          for id  in range(self.batch_size):
            x1,x2,x3=self.__get_in_out()
            x_1_batch[id,]=x1
            x_2_batch[id,]=x2
            x_3_batch[id,]=x3
          return [x_1_batch,x_2_batch,x_3_batch],y_batch

        else:
          x_1_batch=np.empty((self.batch_size,self.image_size,self.image_size,3))
          x_2_batch=np.empty((self.batch_size,self.image_size,self.image_size,3))
          y_batch=np.empty((self.batch_size))         
          for id  in range(self.batch_size):
            x1,x2,y=self.__get_in_out()
            x_1_batch[id,]=x1
            x_2_batch[id,]=x2
            y_batch[id,]=y
          return [x_1_batch,x_2_batch],y_batch
        
    
    def __getitem__(self, index):
        
        x, y = self.__get_data() 
        return x,y
    
    def __len__(self):
        return np.int(self.exa // self.batch_size)


In [ ]:
def read_images(dataset_path):
        imagepaths, labels = list(), list()

        # An ID will be affected to each sub-folders by alphabetical order
        label = 0

        classes = sorted(os.walk(dataset_path).__next__()[1])
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            print(walk)
            # Add each image to the training set
            for sample in walk[2][:50]:
                # Only keeps jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
        num_of_labels=label+1
        return imagepaths, np.array(labels)



In [ ]:
def make_pairs(images, labels):

    pairImages = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]

        idxB = np.random.choice(idx[label])
        posImage = images[idxB]

        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        negIdx = np.where(labels != label)[0]
        negImage = images[np.random.choice(negIdx)]
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
    return (np.array(pairImages), np.array(pairLabels))

In [ ]:
def imread_array(imagesPath,imageShape):
    images= list()
    sizeArray=imagesPath.shape
    imageShape=imageShape[0:2]
    for row in range (sizeArray[0]):
        imageA=cv2.imread(imagesPath[row][0])
        imageA=cv2.resize(imageA,imageShape)
        imageB=cv2.imread(imagesPath[row][1])
        imageB=cv2.resize(imageB,imageShape)
        images.append([imageA, imageB])
    return np.array(images)        
        

In [ ]:
from collections import defaultdict
t = defaultdict(lambda: list)

In [ ]:
def createPredictionSet(data, labels, element_for_class=10):
    unique_labels = set(labels)
    image_for_label = defaultdict(lambda: [])
    for i,img in enumerate(data):
        if len(image_for_label[f"{labels[i]}"]) < element_for_class:
            image_for_label[f"{labels[i]}"].append(img)
        if sum([len(v) for v in image_for_label.values()]) >= len(unique_labels) * element_for_class:
            break
    return image_for_label

In [ ]:
!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia

In [ ]:
!unzip -qq chest-xray-covid19-pneumonia.zip

In [ ]:
# !rm -r /content/Data/test/PNEUMONIA/
# !rm -r /content/Data/train/PNEUMONIA/

In [ ]:
(trainX, trainY) = read_images(r"/content/Data/train")
(testX, testY) = read_images(r"/content/Data/test")
(pairTrainPaths, labelTrain) = make_pairs(trainX, trainY)
(pairTestPaths, labelTest) = make_pairs(testX, testY)
pairTrain=imread_array(pairTrainPaths,IMG_SHAPE)
pairTest=imread_array(pairTestPaths,IMG_SHAPE)
images = []

In [ ]:
for i in np.random.choice(np.arange(0, len(pairTrain)), size=(49,)):    
    imageA=cv2.cvtColor(pairTrain[i][0], cv2.COLOR_BGR2GRAY)
    imageA =cv2.resize(imageA, (28,28))
    imageB =cv2.cvtColor(pairTrain[i][1], cv2.COLOR_BGR2GRAY)
    imageB=cv2.resize(imageB,(28,28))
    
    label = labelTrain[i]
    # to make it easier to visualize the pairs and their positive or
    # negative annotations, we're going to "pad" the pair with four
    # pixels along the top, bottom, and right borders, respectively
    output = np.zeros((36, 60), dtype="uint8")
    pair = np.hstack([imageA, imageB])
    output[4:32, 0:56] = pair
    # set the text label for the pair along with what color we are
    # going to draw the pair in (green for a "positive" pair and
    # red for a "negative" pair)
    text = "neg" if label[0] == 0 else "pos"
    color = (0, 0, 255) if label[0] == 0 else (0, 255, 0)
    # create a 3-channel RGB image from the grayscale pair, resize
    # it from 60x36 to 96x51 (so we can better see it), and then
    # draw what type of pair it is on the image
    vis = cv2.merge([output] * 3)
    vis = cv2.resize(vis, (96, 51), interpolation=cv2.INTER_LINEAR)
    cv2.putText(vis, text, (2, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
        color, 2)
    # add the pair visualization to our list of output images
    images.append(vis)

In [ ]:
from google.colab.patches import cv2_imshow
# construct the montage for the images
montage = build_montages(images, (96, 51), (7, 7))[0]
# show the output montage
cv2_imshow( montage)
cv2.waitKey(0)

In [ ]:
def distance(vectors):
    (featsA, featsB) = vectors
    cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
    out=(cosine_loss(featsA, featsB)+1)/5
    return out

In [ ]:
def plot_training(H, plotPath):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [ ]:
def contrastive_loss_(y, preds, margin=20):

    y = tf.cast(y, preds.dtype)

    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [ ]:
def contrastive_loss(y, preds, margin=1):

    y = tf.cast(y, preds.dtype)

    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [ ]:

def Triplet_loss(vectors):
    margin=1
    (featsA, featsB,featsC) = vectors
    D1=Lambda(distance)([featsA, featsB])
    D2=Lambda(distance)([featsA, featsC])
    loss=K.maximum(0.0,margin+D1-D2)
    return loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
imgA = tf.keras.Input(shape=IMG_SHAPE)
imgB = tf.keras.Input(shape=IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance_out =Lambda(distance)([featsA, featsB])
contrastive_model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance_out)
contrastive_model_ = tf.keras.Model(inputs=[imgA, imgB], outputs=distance_out)

# contrastive_model = tf.keras.models.load_model(con_MODEL_PATH,custom_objects={'contrastive_loss':contrastive_loss})
# contrastive_model_ = tf.keras.models.load_model(con_MODEL_PATH_,custom_objects={'contrastive_loss_':contrastive_loss_})

contrastive_model.compile(loss=contrastive_loss, optimizer="adam")

contrastive_model_.compile(loss=contrastive_loss_, optimizer="adam")

imgA_1 = tf.keras.Input(shape=IMG_SHAPE)
imgB_1 = tf.keras.Input(shape=IMG_SHAPE)
imgC_1 = tf.keras.Input(shape=IMG_SHAPE)

featsA_1 = featureExtractor(imgA_1)
featsB_1 = featureExtractor(imgB_1)
featsC_1 = featureExtractor(imgC_1)

LOSS =Lambda(Triplet_loss)([featsA_1, featsB_1,featsC_1])
Triplet_model = tf.keras.Model(inputs=[imgA_1, imgB_1,imgC_1], outputs=LOSS)
def loss_(gt,est):
  out=est-gt
  return out
opt=tf.keras.optimizers.Adam(
    beta_1=0.7,
    beta_2=0.8)
# Triplet_model = tf.keras.models.load_model(tri_MODEL_PATH,custom_objects={'loss_':loss_})
Triplet_model.compile(loss=loss_, optimizer=opt)

In [ ]:
tf.keras.utils.plot_model(contrastive_model, show_shapes=True,rankdir='LB')


In [ ]:
tf.keras.utils.plot_model(Triplet_model, show_shapes=True,rankdir='LB')


In [ ]:
image_len_wid=224
BATCH_SIZE = 32*4
EPOCHS = 16

train_contras_gen=DataGen(path='/content/Data/train',batch_size=BATCH_SIZE,image_size=image_len_wid,exa=25*32)
test_contras_gen=DataGen(path='/content/Data/test',batch_size=BATCH_SIZE,image_size=image_len_wid,exa=10*32)

train_Triplet_gen=DataGen(path='/content/Data/train',batch_size=BATCH_SIZE,image_size=image_len_wid,loss_type='Triplet',exa=25*32)
test_Triplet_gen=DataGen(path='/content/Data/test',batch_size=BATCH_SIZE,image_size=image_len_wid,loss_type='Triplet',exa=10*32)


In [ ]:

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=tri_MODEL_PATH, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

#Triplet_model.trainable=True
Triplet_history = Triplet_model.fit(
    train_Triplet_gen,
    validation_data=test_Triplet_gen,
    validation_freq=2,
    callbacks=checkpoint,
    epochs=EPOCHS)
 

In [ ]:
 
# plot the training history
plot_training(Triplet_history, tri_PLOT_PATH)


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=con_MODEL_PATH, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
contrastive_history = contrastive_model.fit(
    train_contras_gen,
    validation_data=test_contras_gen,
    validation_freq=2,
    callbacks=checkpoint,
    epochs=16)


In [ ]:

# plot the training history
plot_training(contrastive_history, con_PLOT_PATH)

In [ ]:
checkpoint_ = tf.keras.callbacks.ModelCheckpoint(filepath=con_MODEL_PATH_, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
contrastive_history_ = contrastive_model_.fit(
    train_contras_gen,
    validation_data=test_contras_gen,
    validation_freq=2,
    callbacks=checkpoint_,
    epochs=16)


In [ ]:

# plot the training history
plot_training(contrastive_history_, con_PLOT_PATH_)

In [ ]:
con_siamese_model=contrastive_model.get_layer(index=2)
con_siamese_model_=contrastive_model_.get_layer(index=2)

tri_siamese_model=Triplet_model.get_layer(index=3)



In [ ]:
image=cv2.imread('/content/Data/test/COVID19/COVID19(460).jpg')
print(image.shape)
image=image/255
image=cv2.resize(image,(224,224))
image=np.expand_dims(image,axis=0)
print(image.shape)

In [ ]:
x=con_siamese_model(image)
print(x.numpy())

In [ ]:
def createImagePairWithPredictionSet(image, prediction_set, image_shape=(224, 224)):
    prediction_set_img = defaultdict(lambda: [])
    prediction_set_img_pairs = defaultdict(lambda: [])
    for key, value in prediction_set.items():
        for img in value:
            new_img = cv2.imread(img)
            new_img = cv2.resize(new_img, image_shape)
            # new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
            prediction_set_img[key].append(new_img)
            
    if isinstance(image, str):
        test_img = cv2.imread(image)
        test_img = cv2.resize(test_img, image_shape)
        # test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    else:
        image = np.squeeze(image)
        if image.shape != (224, 224):
            test_img = cv2.resize(image, image_shape)
            # test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
        else:
            test_img = image  
    
    test_img = np.expand_dims(test_img, axis=0)
    for key, value in prediction_set_img.items():
        for img in value:
            img = np.expand_dims(img, axis=0)

            prediction_set_img_pairs[key].append([test_img, img])
    
    return prediction_set_img_pairs

In [ ]:
prediction_data = createPredictionSet(trainX, trainY, element_for_class=20)
predictions = []
for i, test_img_path in enumerate(testX):
    prediction_set_img_pairs = createImagePairWithPredictionSet(test_img_path, prediction_data)
    highest_value_per_label = {}
    for key, value in prediction_set_img_pairs.items():
        key_output = []
        for pair in value:
            preds = contrastive_model.predict(pair)
            key_output.append(preds[0][0])
        highest_value_per_label[key] = np.min(key_output)
    predictions.append(int(min(highest_value_per_label, key=highest_value_per_label.get)))

In [ ]:
accuracy_score(testY, predictions)

In [ ]:
def list_full_paths(directory):
  return [os.path.join(directory, file) for file in os.listdir(directory)]

def read_img(list_,wid_len,cha):
  num=len(list_)
  patch=np.empty((num,wid_len,wid_len,cha))
  for i in range(num):
    img=np.asarray(cv2.imread(list_[i]))/255
    patch[i]=np.resize(img,(wid_len,wid_len,cha))

  return patch
example_per_class=400


In [ ]:




example_per_class=400
list_train_COV=list_full_paths('/content/Data/train/COVID19')
cov_example_path=sample(list_train_COV, example_per_class)
cov_example=read_img(cov_example_path,image_len_wid,3)

list_train_NON=list_full_paths('/content/Data/train/NORMAL')
non_example_path=sample(list_train_NON, example_per_class)
non_example=read_img(non_example_path,image_len_wid,3)

list_train_PNEUMONIA=list_full_paths('/content/Data/train/PNEUMONIA')
PNEUMONIA_example_path=sample(list_train_PNEUMONIA, example_per_class)
PNEUMONIA_example=read_img(PNEUMONIA_example_path,image_len_wid,3)


cov_vectors=tri_siamese_model.predict(cov_example)
cov_label=np.full((np.shape(cov_vectors)[0]), 0, dtype=int)


PNEUMONIA_vectors=tri_siamese_model.predict(PNEUMONIA_example)
p_label=np.full((np.shape(PNEUMONIA_vectors)[0]), 1, dtype=int)

non_vectors=tri_siamese_model.predict(non_example)
non_label=np.full((np.shape(non_vectors)[0]), 2, dtype=int)

x=np.concatenate((cov_vectors, PNEUMONIA_vectors,non_vectors), axis=0)
y=np.concatenate((cov_label, p_label,non_label), axis=0)

clf = svm.LinearSVC(tol=.000001)
clf.fit(x, y)
yy=clf.predict(x)

o=yy-y
print((o.size-np.count_nonzero(o))/1200)

non_test_path=list_full_paths('/content/Data/test/NORMAL')
cov_test_path=list_full_paths('/content/Data/test/COVID19')
PNEUMONIA_test_path=list_full_paths('/content/Data/test/PNEUMONIA')

test_non=read_img(non_test_path,image_len_wid,3)
test_fea=tri_siamese_model.predict(test_non)
non_pre=clf.predict(test_fea)

unique, counts = np.unique(non_pre, return_counts=True)
non_=dict(zip(unique, counts))
print(non_)

test_cov=read_img(cov_test_path,image_len_wid,3)
test_cov_fea=tri_siamese_model.predict(test_cov)
cov_pre=clf.predict(test_cov_fea)

unique, counts = np.unique(cov_pre, return_counts=True)
cov_=dict(zip(unique, counts))
print(cov_)


test_p=read_img(PNEUMONIA_test_path,image_len_wid,3)
test_p_fea=tri_siamese_model.predict(test_p)
pne_pre=clf.predict(test_p_fea)

unique, counts = np.unique(pne_pre, return_counts=True)
pne_=dict(zip(unique, counts))
print(pne_)

C=np.empty((3,3))
C[1]=[pne_[0],pne_[1],pne_[2]]/(pne_[0]+pne_[1]+pne_[2])
C[0]=[cov_[0],cov_[1],cov_[2]]/(cov_[0]+cov_[1]+cov_[2])
C[2]=[non_[0],non_[1],non_[2]]/(non_[0]+non_[1]+non_[2])
plt.imshow(C, cmap='gray', interpolation='nearest')
plt.show()


In [ ]:
print(C)
print("acc= ",(C[0,0]+C[1,1]+C[2,2])/np.sum(C))

In [ ]:




#example_per_class=400
#list_train_COV=list_full_paths('/content/Data/train/COVID19')
#cov_example_path=sample(list_train_COV, example_per_class)
#cov_example=read_img(cov_example_path,image_len_wid,3)

# list_train_NON=list_full_paths('/content/Data/train/NORMAL')
# non_example_path=sample(list_train_NON, example_per_class)
# non_example=read_img(non_example_path,image_len_wid,3)

# list_train_PNEUMONIA=list_full_paths('/content/Data/train/PNEUMONIA')
# PNEUMONIA_example_path=sample(list_train_PNEUMONIA, example_per_class)
# PNEUMONIA_example=read_img(PNEUMONIA_example_path,image_len_wid,3)


cov_vectors=con_siamese_model.predict(cov_example)
cov_label=np.full((np.shape(cov_vectors)[0]), 0, dtype=int)


PNEUMONIA_vectors=con_siamese_model.predict(PNEUMONIA_example)
p_label=np.full((np.shape(PNEUMONIA_vectors)[0]), 1, dtype=int)

non_vectors=con_siamese_model.predict(non_example)
non_label=np.full((np.shape(non_vectors)[0]), 2, dtype=int)

x=np.concatenate((cov_vectors, PNEUMONIA_vectors,non_vectors), axis=0)
y=np.concatenate((cov_label, p_label,non_label), axis=0)

clf = svm.LinearSVC(tol=.000001)
clf.fit(x, y)
yy=clf.predict(x)

o=yy-y
print((o.size-np.count_nonzero(o))/1200)

# non_test_path=list_full_paths('/content/Data/test/NORMAL')
# cov_test_path=list_full_paths('/content/Data/test/COVID19')
# PNEUMONIA_test_path=list_full_paths('/content/Data/test/PNEUMONIA')

#test_non=read_img(non_test_path,image_len_wid,3)
test_fea=con_siamese_model.predict(test_non)
non_pre=clf.predict(test_fea)

unique, counts = np.unique(non_pre, return_counts=True)
non_=dict(zip(unique, counts))
print(non_)

#test_cov=read_img(cov_test_path,image_len_wid,3)
test_cov_fea=con_siamese_model.predict(test_cov)
cov_pre=clf.predict(test_cov_fea)

unique, counts = np.unique(cov_pre, return_counts=True)
cov_=dict(zip(unique, counts))
print(cov_)


#test_p=read_img(PNEUMONIA_test_path,image_len_wid,3)
test_p_fea=con_siamese_model.predict(test_p)
pne_pre=clf.predict(test_p_fea)

unique, counts = np.unique(pne_pre, return_counts=True)
pne_=dict(zip(unique, counts))
print(pne_)

C=np.empty((3,3))
C[1]=[pne_[0],pne_[1],pne_[2]]
C[0]=[cov_[0],cov_[1],cov_[2]]
C[2]=[non_[0],non_[1],non_[2]]
plt.imshow(C, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
print(C)
print("acc= ",(C[0,0]+C[1,1]+C[2,2])/np.sum(C))

In [ ]:




#example_per_class=400
#list_train_COV=list_full_paths('/content/Data/train/COVID19')
#cov_example_path=sample(list_train_COV, example_per_class)
#cov_example=read_img(cov_example_path,image_len_wid,3)

# list_train_NON=list_full_paths('/content/Data/train/NORMAL')
# non_example_path=sample(list_train_NON, example_per_class)
# non_example=read_img(non_example_path,image_len_wid,3)

# list_train_PNEUMONIA=list_full_paths('/content/Data/train/PNEUMONIA')
# PNEUMONIA_example_path=sample(list_train_PNEUMONIA, example_per_class)
# PNEUMONIA_example=read_img(PNEUMONIA_example_path,image_len_wid,3)


cov_vectors=con_siamese_model_.predict(cov_example)
cov_label=np.full((np.shape(cov_vectors)[0]), 0, dtype=int)


PNEUMONIA_vectors=con_siamese_model_.predict(PNEUMONIA_example)
p_label=np.full((np.shape(PNEUMONIA_vectors)[0]), 1, dtype=int)

non_vectors=con_siamese_model_.predict(non_example)
non_label=np.full((np.shape(non_vectors)[0]), 2, dtype=int)

x=np.concatenate((cov_vectors, PNEUMONIA_vectors,non_vectors), axis=0)
y=np.concatenate((cov_label, p_label,non_label), axis=0)

clf = svm.LinearSVC(tol=.000001)
clf.fit(x, y)
yy=clf.predict(x)

o=yy-y
print((o.size-np.count_nonzero(o))/1200)

# non_test_path=list_full_paths('/content/Data/test/NORMAL')
# cov_test_path=list_full_paths('/content/Data/test/COVID19')
# PNEUMONIA_test_path=list_full_paths('/content/Data/test/PNEUMONIA')

#test_non=read_img(non_test_path,image_len_wid,3)
test_fea=con_siamese_model_.predict(test_non)
non_pre=clf.predict(test_fea)

unique, counts = np.unique(non_pre, return_counts=True)
non_=dict(zip(unique, counts))
print(non_)

#test_cov=read_img(cov_test_path,image_len_wid,3)
test_cov_fea=con_siamese_model_.predict(test_cov)
cov_pre=clf.predict(test_cov_fea)

unique, counts = np.unique(cov_pre, return_counts=True)
cov_=dict(zip(unique, counts))
print(cov_)


#test_p=read_img(PNEUMONIA_test_path,image_len_wid,3)
test_p_fea=con_siamese_model_.predict(test_p)
pne_pre=clf.predict(test_p_fea)

unique, counts = np.unique(pne_pre, return_counts=True)
pne_=dict(zip(unique, counts))
print(pne_)

C=np.empty((3,3))
C[1]=[pne_[0],pne_[1],pne_[2]]
C[0]=[cov_[0],cov_[1],cov_[2]]
C[2]=[non_[0],non_[1],non_[2]]
plt.imshow(C, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
print(C)
print("acc= ",(C[0,0]+C[1,1]+C[2,2])/np.sum(C))

In [ ]:




#example_per_class=400
#list_train_COV=list_full_paths('/content/Data/train/COVID19')
#cov_example_path=sample(list_train_COV, example_per_class)
#cov_example=read_img(cov_example_path,image_len_wid,3)

# list_train_NON=list_full_paths('/content/Data/train/NORMAL')
# non_example_path=sample(list_train_NON, example_per_class)
# non_example=read_img(non_example_path,image_len_wid,3)

# list_train_PNEUMONIA=list_full_paths('/content/Data/train/PNEUMONIA')
# PNEUMONIA_example_path=sample(list_train_PNEUMONIA, example_per_class)
# PNEUMONIA_example=read_img(PNEUMONIA_example_path,image_len_wid,3)


cov_vectors=np.concatenate((con_siamese_model.predict(cov_example),con_siamese_model_.predict(cov_example),tri_siamese_model.predict(cov_example)),axis=1)
cov_label=np.full((np.shape(cov_vectors)[0]), 0, dtype=int)


PNEUMONIA_vectors=np.concatenate((con_siamese_model.predict(PNEUMONIA_example),con_siamese_model_.predict(PNEUMONIA_example),tri_siamese_model.predict(PNEUMONIA_example)),axis=1)
p_label=np.full((np.shape(PNEUMONIA_vectors)[0]), 1, dtype=int)

non_vectors=np.concatenate((con_siamese_model.predict(non_example),con_siamese_model_.predict(non_example),tri_siamese_model.predict(non_example)),axis=1)
non_label=np.full((np.shape(non_vectors)[0]), 2, dtype=int)

x=np.concatenate((cov_vectors, PNEUMONIA_vectors,non_vectors), axis=0)
y=np.concatenate((cov_label, p_label,non_label), axis=0)

clf = svm.LinearSVC(tol=.000001)
clf.fit(x, y)
yy=clf.predict(x)

o=yy-y
print((o.size-np.count_nonzero(o))/1200)

# non_test_path=list_full_paths('/content/Data/test/NORMAL')
# cov_test_path=list_full_paths('/content/Data/test/COVID19')
# PNEUMONIA_test_path=list_full_paths('/content/Data/test/PNEUMONIA')

#test_non=read_img(non_test_path,image_len_wid,3)
test_fea=np.concatenate((con_siamese_model.predict(test_non),con_siamese_model_.predict(test_non),tri_siamese_model.predict(test_non)),axis=1)
non_pre=clf.predict(test_fea)

unique, counts = np.unique(non_pre, return_counts=True)
non_=dict(zip(unique, counts))
print(non_)

#test_cov=read_img(cov_test_path,image_len_wid,3)
test_cov_fea=np.concatenate((con_siamese_model.predict(test_cov),con_siamese_model_.predict(test_cov),tri_siamese_model.predict(test_cov)),axis=1)
cov_pre=clf.predict(test_cov_fea)

unique, counts = np.unique(cov_pre, return_counts=True)
cov_=dict(zip(unique, counts))
print(cov_)


#test_p=read_img(PNEUMONIA_test_path,image_len_wid,3)
test_p_fea=np.concatenate((con_siamese_model.predict(test_p),con_siamese_model_.predict(test_p),tri_siamese_model.predict(test_p)),axis=1)
pne_pre=clf.predict(test_p_fea)

unique, counts = np.unique(pne_pre, return_counts=True)
pne_=dict(zip(unique, counts))
print(pne_)

C=np.empty((3,3))
C[1]=[pne_[0],pne_[1],pne_[2]]
C[0]=[cov_[0],cov_[1],cov_[2]]
C[2]=[non_[0],non_[1],non_[2]]
plt.imshow(C, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
print(C)
print("acc= ",(C[0,0]+C[1,1]+C[2,2])/np.sum(C))